In [5]:
import requests
import pandas as pd
import numpy as np
import os
import requests
from dotenv import load_dotenv



In [7]:
from dune_client.client import DuneClient

load_dotenv()
dune = DuneClient("$DUNE_WHALES_MOVES")
query_result = dune.get_latest_result(6184996)
limit= 1500

HTTPError: 401 Client Error: Unauthorized for url: https://api.dune.com/api/v1/query/6184996/results?limit=1

In [9]:
API_KEY = os.getenv("DUNE_WHALES_API")  # Set this in your .env or environment
QUERY_ID = 6184996

url = f"https://api.dune.com/api/v1/query/{6184996}/execute"
headers = {"x-dune-api-key": API_KEY}

response = requests.post(url, headers=headers)
data = response.json()

print(data)

# Check status
print(data["status"])
# Get results
results = data["result"]["rows"]

{'execution_id': '01K9S410H8F5F5VW48WHQB9HCZ', 'state': 'QUERY_STATE_PENDING'}


KeyError: 'status'

In [17]:
import time
import pandas as pd

execution_id = "01K9S410H8F5F5VW48WHQB9HCZ"
results_url = f"https://api.dune.com/api/v1/execution/{execution_id}/results"

while True:
    results_response = requests.get(results_url, headers=headers)
    results_data = results_response.json()
    print("State:", results_data.get("state"))

    if results_data.get("state") == "QUERY_STATE_COMPLETED":
        print("✅ Query complete!")
        break
    elif results_data.get("state") == "QUERY_STATE_FAILED":
        raise RuntimeError("❌ Query failed:", results_data)
    else:
        time.sleep(10)

# Extract results into a dataframe
rows = results_data["result"]["rows"]
df = pd.DataFrame(rows)
print(df.head())


State: QUERY_STATE_COMPLETED
✅ Query complete!
   avg_whale_tx_size_usd  block_date  total_whale_volume_usd  \
0           1.102311e+06  2025-11-11            2.162734e+09   
1           1.355481e+06  2025-11-10            9.468031e+09   
2           7.837350e+05  2025-11-09            5.446959e+09   
3           9.275734e+05  2025-11-08            3.847574e+09   
4           1.360536e+06  2025-11-07            1.222986e+10   

   unique_whale_senders  whale_exchange_deposits_usd  \
0                   738                 1.789810e+08   
1                  2037                 6.341238e+08   
2                  1601                 3.789824e+08   
3                  1286                 2.375570e+08   
4                  2701                 6.187801e+08   

   whale_exchange_withdrawals_usd  whale_net_exchange_flow_usd  whale_tx_count  
0                    2.954776e+08                 1.164966e+08            1449  
1                    8.458045e+08                 2.116807e+08       

In [18]:
df.head(5)

,avg_whale_tx_size_usd,block_date,total_whale_volume_usd,unique_whale_senders,whale_exchange_deposits_usd,whale_exchange_withdrawals_usd,whale_net_exchange_flow_usd,whale_tx_count
0,1.102311e+06,2025-11-11,2.162734e+09,738,1.789810e+08,2.954776e+08,1.164966e+08,1449
1,1.355481e+06,2025-11-10,9.468031e+09,2037,6.341238e+08,8.458045e+08,2.116807e+08,5023
2,7.837350e+05,2025-11-09,5.446959e+09,1601,3.789824e+08,4.754272e+08,9.644479e+07,4092
3,9.275734e+05,2025-11-08,3.847574e+09,1286,2.375570e+08,2.631461e+08,2.558916e+07,2916
4,1.360536e+06,2025-11-07,1.222986e+10,2701,6.187801e+08,5.767914e+08,-4.198871e+07,6159


In [23]:
df.dtypes

avg_whale_tx_size_usd             float64
block_date                         object
total_whale_volume_usd            float64
unique_whale_senders                int64
whale_exchange_deposits_usd       float64
whale_exchange_withdrawals_usd    float64
whale_net_exchange_flow_usd       float64
whale_tx_count                      int64
dtype: object

In [28]:
import requests
import pandas as pd

# Replace with your token id and API key
token_id = "ethereum"  # e.g. "ethereum", "solana", or your token id on Coingecko
api_key = "COINGECKO_API_KEY"

headers = {
    "accept": "application/json",
    "x-cg-pro-api-key": api_key
}

url = f"https://pro-api.coingecko.com/api/v3/coins/{token_id}/market_chart?vs_currency=usd&days=max"

resp = requests.get(url, headers=headers)
data = resp.json()

# Convert to DataFrame
prices = pd.DataFrame(data["prices"], columns=["timestamp", "price_usd"])
prices["block_date"] = pd.to_datetime(prices["timestamp"], unit="ms").dt.date

print(prices.head())


KeyError: 'prices'

In [19]:
# ### Fetch Ethereum Price Data
# Get daily Ethereum price data for the last 5 years from CoinGecko.
# Define CoinGecko endpoint and parameters

coingecko_url = "https://api.coingecko.com/api/v3/coins/ethereum/market_chart"
params = {
    "vs_currency": "usd",
    "days": "1360",
    "interval": "daily"
}

# Fetch price data
cg_response = requests.get(coingecko_url, params=params)
cg_data = cg_response.json()

# Convert to DataFrame
eth_prices = pd.DataFrame(cg_data['prices'], columns=['timestamp', 'eth_price'])
eth_prices['timestamp'] = pd.to_datetime(eth_prices['timestamp'], unit='ms')
eth_prices['date'] = eth_prices['timestamp'].dt.date
eth_prices = eth_prices[['date', 'eth_price']]

KeyError: 'prices'